In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
pinecone_api_key = "pinecone-api-key"

In [4]:
PINECONE_API_ENV = "index-name"

In [5]:
# Extract data from PDF
def load_pdf(pdf_path):
    loader = DirectoryLoader(pdf_path,
                   glob="*.pdf",
                   loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [6]:
model = "llama-2-7b-chat.ggmlv3.g4_0.bin"

In [ ]:
extracted_data = load_pdf("data/medical_book.pdf")

# Create text chunks

In [7]:
def text_slpite(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlab=20)
    text_chunks = text_splitter.split_socuments(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_slpite(extracted_data)
print(f"Length of my chunks: {len(text_chunks)}")

In [8]:
# Download Embeding model
def download_huggingface_embedding():
    embeding = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeding

In [ ]:
embedding_text = download_huggingface_embedding()

In [ ]:
query_results = embedding.embed_query("Hello World")
print(f"Length : {len(query_results)}")

In [ ]:
pinecone.init(api_key=PINECONE_API_KEY,
             environment=PINECONE_API_ENV)

indes_name = "index-name"

# Creating Embedding for Each of the Text Chunks and Storing
doc_search = Pinecone.from_texts([t.page_content for t in text_chunks], embedding_text, index_name = indes_name)

In [ ]:
# if we already have an index we can load it like this
doc_search = Pinecone.from_existing_index(index_name, embedding_text)

query = "What are Allergies"

docs = doc_search.similarity_search(query, k=3)
print(f"Results : {docs}")

In [ ]:
prompt_template = """

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context : {context}
Question : {question}

Only return the helpful answer belos and nothing else.
Helpful answer:


"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwarges = {"prompt":PROMPT}

In [ ]:
llm = CTransformers(model = model,
                   model_type="llama",
                   config={"max_new_tokens":512,
                          "temperature":0.8})

In [ ]:
question_answer = RetrivalQA.from_chain_type(
    llm=llm,
    chain_type="styff",
    retriver = doc_search.as_retriever(search_kwargs={"k":2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt: ")
    results = question_answer({"query":user_input})
    print("Response : ", results["result"])